In [ ]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [ ]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        for attr, value in self.__dict__.items():
            print("\nAttribute: " + str(attr))
            print("Value: {0}".format(value))

In [ ]:
print(m.acos(4/5)/m.pi)

In [ ]:
nArray = np.array([[1,2], [1,3], [1,4]])
nodes = Input("Node numbers", nArray, "None")
nodes.printValues()

# list of lengths corresponding to each element
# the length of element n is contained in lengt

lArray = np.array([1600, 2000, 1200])
lengths = Input("Lengths", lArray, "[m]")
lengths.printValues()

# list of angles relative to positive x axis for each element
aArray = np.array([1, (1 + m.acos(4/5)/m.pi), 1.5])
aArray *= m.pi
angles = Input("Element Angles (relative to the positive x axis)", aArray, "[rad]")
angles.printValues()

Constants Inputs
<br>Units are : MPa, mm, mm2 , N/mm, N

In [ ]:
E = 120000
A = 500
P1x = 25000


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [ ]:
numElems = len(nodes.value)
print(numElems)
numNodes = nodes.value.max()
print(numNodes)

displacements in mm, two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.
<br><br>this setup is advantageous over a numnodes x 2 array as it allows for easier value access later, when many sub-arrays will be required.


In [ ]:
uvList = [None]*(numNodes*2)
uv_BCs = np.array([[2,0,0],[3,0,0],[4,0,0]])
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

In [ ]:
adjust_array(uvList, uv_BCs, "displacements")

print("u & v list:")
print(uvList)

Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [ ]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

force_BCs = np.array([[1, P1x, 0],[2, None, None], [3, None, None], [4, None, None]])
print(len(force_BCs))
loadedNodes = 0
for i in range(len(force_BCs)):
    if force_BCs[i,1] or force_BCs[i,2] != None:
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print("number of nodes with concentrated loads: {0}".format(loadedNodes))
print("at nodes: {0}".format(nodesWithLoads))

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print("force list: ")
print(forceList)

Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [ ]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(globalK.shape)

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    barConstant[i] = (E*A)/lengths.value[i]

print("Bar Constants = ")
print(barConstant)

## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [ ]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    print("\nk matrix for element {1}: \n{0}".format(kMatrix, (i+1)))
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print("Element " + str(i+1) + " starting at node: " + str(startNode) + " and ending at node: " + str(endNode))
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print("\nGlobal K matrix: \n{0}".format(globalK))
unmodified_globalK = globalK.copy()
check_Symmetric(globalK)

In [ ]:
#print(inv(globalK))
u = np.linalg.inv(globalK).dot(forceList)
print(u)

In [ ]:
j = 0
while j < numNodes*2:
    print(str(j))
    if j in nodesWithLoads:
        j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    elif uvList[j] == 0:
        print("u = 0 at row: " + str(j))
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print("none detected at row " + str(j))
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print("row " + str(j) + " altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

In [ ]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

node = 1

uFinal = np.matmul(globalK_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print("u{0} = {1}".format((node), uFinal[i]))
    else:
        print("v{0} = {1}".format((node), uFinal[i]))
        node += 1 

Forces can be found from [K]*[u]

In [ ]:
forcesFinal = np.matmul(unmodified_globalK, uFinal)
for i in range(len(forceList)):
    if forceList[i] != forcesFinal[i] and forceList[i] != 0 and forceList[i] != None:
        forcesFinal[i] = forceList[i]
print(forcesFinal)

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print("fx{0} = {1}".format((node), forcesFinal[i]))
    else:
        print("fy{0} = {1}".format((node), forcesFinal[i]))
        node += 1 


## Finding Stresses

In [ ]:
stressFinal = np.zeros((numElems, 1))
print(stressFinal)
for i in range(numElems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print("Element {0} starts at node: {1} and ends at node: {2}".format((i+1), startNode, endNode))
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print("adjusted start node: {0} adjusted end node: {1}".format(Sn, En))
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print("trigMatrix = \n{0}".format(trigMatrix))
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print("local u matrix: \n{0}".format(local_uMatrix))
    localLength = np.array([[(-1/lengths.value[i]), 1/lengths.value[i]]])
    print("local length matrix: \n{0}".format(localLength))

    indStress = (trigMatrix @ local_uMatrix)
    indStress = localLength @ indStress
    indStress *= E

    stressFinal[i] = indStress
    
    
    print("Stress in element = {0}".format(indStress))

    

In [ ]:
print(stressFinal)